<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Extracted-information" data-toc-modified-id="Extracted-information-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Extracted information</a></span></li><li><span><a href="#Missing-reports" data-toc-modified-id="Missing-reports-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Missing reports</a></span><ul class="toc-item"><li><span><a href="#Mandatory-information" data-toc-modified-id="Mandatory-information-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Mandatory information</a></span></li></ul></li><li><span><a href="#Incomplete-information-processing" data-toc-modified-id="Incomplete-information-processing-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Incomplete information processing</a></span></li><li><span><a href="#About-date-normalization" data-toc-modified-id="About-date-normalization-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>About date normalization</a></span></li></ul></li><li><span><a href="#Import-and-load" data-toc-modified-id="Import-and-load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import and load</a></span></li><li><span><a href="#Parse-XML-files" data-toc-modified-id="Parse-XML-files-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parse XML files</a></span><ul class="toc-item"><li><span><a href="#Sanity-check" data-toc-modified-id="Sanity-check-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Sanity check</a></span></li></ul></li><li><span><a href="#Merge-files" data-toc-modified-id="Merge-files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merge files</a></span></li><li><span><a href="#Make-DataFrame" data-toc-modified-id="Make-DataFrame-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Make DataFrame</a></span><ul class="toc-item"><li><span><a href="#Remove-duplicate-case_id-and-report_id" data-toc-modified-id="Remove-duplicate-case_id-and-report_id-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Remove duplicate case_id and report_id</a></span></li><li><span><a href="#Add-receiptdate" data-toc-modified-id="Add-receiptdate-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Add receiptdate</a></span></li></ul></li></ul></div>

# Introduction
This script extract information from each xml files and save as dictionary in pickle format. The files are extracted in quater-wise. 

## Extracted information
In this file, each report is extracted as one recording (assigned a dictionary key). The recording could contains multiple reactions or drugs. The data is formed as `dic = {key:value}` where key is the order/number of reports/recorddings, and the value is a list contains informative elements: [version, report_id, case_id, country, qualify, serious, serious_subtype_1, serious_subtype_2, serious_subtype_3, serious_subtype_4, serious_subtype_5, serious_subtype_6, receivedate, receiptdate, age, gender, weight, reaction_list, drug_list].

<ol>
    <li>Admin:</li> *version, report_id, case_id, country, qualify, serious, serious_subtype_1, serious_subtype_2, serious_subtype_3, serious_subtype_4, serious_subtype_5, serious_subtype_6, receivedate, receiptdate* are admin information. Index: 0-13               
    <li>Demograph:</li> *age, gender, weight,* Index: 14-16
    <li>Reaction:</li> reaction_list, (index:17); reaction_list contains arbitrary reactions, each reaction is a list with three elements (0-2): [PT_code, PT, outcome]
    <li>Drug:</li> drug_list, (index:18); drug_list contains arbitrary drugs, each drug is a list with 16 elements (0-15): [char, product, dorse, unit, drugseparatedosagenumb, drugintervaldosageunitnumb,                             drugintervaldosagedefinition, form, route, indication, start_date, end_date, action, readm,                     additional, substance]  
</ol>



## Missing reports
Each subtype (e.g., admin) contains <a href = "#mandatory">mandatory</a> and optional/voluntary information (defined by us). `miss_admin, miss_patient, miss_reaction, miss_drug` represents the number of reports that <i>Mandatory Information</i> missing administrative/patient/reaction/drug information, respectively. The reports with missed mandatory information are not included in our extracted file.

### Mandatory information
<ol>
    <li>Admin:</li> country, qualify, serious                          
    <li>Demograph:</li> age & gender (at least has one item)
    <li>Reaction:</li> in each reaction, the PT (side effect) is mandatory. The reaction_list should contains at least 1 reaction; otherwise, this report will be ignored.
    <li>Drug:</li> in each drug, substance are mandatory. The drug_list should contains at least 1 drug; otherwise, this report will be ignored.
</ol>

## Incomplete information processing

<ul>
    <li>Missing <code>qualify</code> is set as <code>'6'</code> while <code>'0'</code> refers to others (\ie, there is a number but not in [1,6]). </li>
    <li>Missing <code>receivedate</code>, <code>receiptdate</code> are set as <code>'0'</code>.</li>
    <li>Missing <code>age</code>, <code>gender</code>,<code>weight</code> are set as <code>'0'</code>. If <code>age</code> and <code>gender</code> are both missing, ignore this report. gender= '0' indicates unknown sex. </li>
    <li>Missing <code>PT_code</code>, <code>outcome</code> are set as <code>'0'</code> and <code>'6'</code> respectively.  For outcome, 6 refers to 'unknown'</li>
    <li>Missing <code>char</code>, <code>dorse, unit, drugseparatedosagenumb,drugintervaldosageunitnumb, drugintervaldosagedefinition, form</code> are set as <code>'0'</code>.</li>
    <li>Missing <code>product</code> is set as <code>none</code>.</li>
    <li>Missing <code>start_date</code> is set as <code>'0'</code>; Missing <code>end_date</code> is set as <code>receiptdate</code>, or as <code>'0'</code> if <code>receiptdate</code> is also missing.</li>
    <li>Missing <code>action</code> is set as <code>'5'</code>.</li>
    <li>Missing <code>additional</code> is set as <code>'3'</code>.</li>
    <li>Missing <code>readm</code> is set as <code>3</code> which means 'unknown'.</li>
    <li>Missing <code>substance</code> is set as <code>'none'</code>.</li>
</ul>
The specific meanings of each code, please refer to `Code/Files/XML_NTS.pdf`. 

## About date normalization
Since the dates (such as received date, receipt date, drug start/end date) are in different formats (e.g., yyyymmdd, yyyymm, yyyy), we normalize them as follows:
<ul>
    <li>Set the stand/calibrated date as <code>2000-1-1</code> (Day 0) and calculate the difference between current date and the stand date. For example, <code>2000-1-10</code> is normalized to <code>10</code> days.</li>
    <li>All the dates without precise day are regarded as the 1st day of the month; likewise, the dates without specific month are regarded as January. For example, <code>2015</code> in yyyy formate is regarded as <code>2015-1-1</code>. Then it is normalized to <code>5479</code> days. </li>
</ul>




# Import and load

In [1]:
# %load dic_reading.py
import pickle
import numpy as np
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import glob
from tqdm import tqdm
import itertools

from datetime import date, datetime

se_dic = pickle.load(open('../Data/curated/AE_mapping.pk', 'rb'))
drug_dic = pickle.load(open('../Data/curated/drug_dic.pk', 'rb'))

# In this MeDRA_dic, key is string of PT_name, value is a list:
# [PT, PT_name, HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr]
meddra_pd_all = pickle.load(open('../Data/curated/drug_mapping.pk', 'rb'))

In [2]:
# initial setup
def date_normalize(formate, dat): 
    stand_date = date(2000, 1, 1)
    if formate=='102':  # the date is formed as yyyymmdd
        current_date = date(int(dat[:4]), int(dat[4:6]), int(dat[6:8])) 
    elif formate=='610':  # formed as yyyymm
        current_date = date(int(dat[:4]), int(dat[4:6]), 1)
    elif formate=='602':  #formed as yyyy
        current_date = date(int(dat[:4]), 1, 1)
    delta = current_date - stand_date
    return delta.days

def days_to_date(days):
    stand_date = date(2000, 1, 1)

    if int(days)<0:
        days = 1
    
    dt = datetime.fromordinal(int(days))
    return dt.strftime('%Y-%m-%d')

# Parse XML files

In [3]:
# initial setup
def date_normalize(formate, dat): 
    stand_date = date(2000, 1, 1)
    if formate=='102':  # the date is formed as yyyymmdd
        current_date = date(int(dat[:4]), int(dat[4:6]), int(dat[6:8])) 
    elif formate=='610':  # formed as yyyymm  
        current_date = date(int(dat[:4]), int(dat[4:6]), 1)
    elif formate=='602':  #formed as yyyy      
        current_date = date(int(dat[:4]), 1, 1)
    delta = current_date - stand_date
    return delta.days

n_reports = []
miss_count = {}
# To save time, parse 2018-2018  in the first round, then 2018-2021
for yr in range(2013, 2021):

    if yr == 2020:
        qtr_list = [1, 2, 3]
#         qtr_list = [3]
    else:
        qtr_list=[1,2,3,4]
    for qtr in qtr_list:
        qtr_name = str(yr)+'q'+ str(qtr)
        print('I am parsing:',qtr_name)
        
#         """Read data from lab storage""" 
#         /n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/
        lab_storage = '/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/'

        files = lab_storage + qtr_name + '/**/**'
        xml_files = glob.glob(files +"/*.xml", recursive=True)
        unique_files = list(set(xml_files))  # only keep the unique values, remove duplicated files.
        xml_files = unique_files
        xml_files.sort()
        print('find {} files'.format(len(xml_files)))
        print(xml_files)

        root = None
        for xml_file in xml_files:
            print(xml_file)
            data = ET.parse(xml_file).getroot()
            if root is None:
                root = data
            else:
                root.extend(data)
                print('finished merge',xml_file)
        nmb_reports = len(root)
        print(nmb_reports)

        count = 0
        patient_ID = 0
        dic = {}
        
        miss_admin = miss_patient = miss_reaction = miss_drug =0
        for report in tqdm(root.findall('safetyreport')):
            """Administrative Information"""
#             report.find('').text
            try:  # Mandatory Information: report_id
                try:
                    version = report.find('safetyreportversion').text
                except:
                    version = '1'
                    
                report_id = report.find('safetyreportid').text
                
                try:
                    case_id = report.find('companynumb').text
                except:
                    case_id = '0'  # unknown case id
                    
                try:
                    country = report.find('primarysource')[0].text
                except:
                    country = 'unknown'          

                    
                if country =='COUNTRY NOT SPECIFIED':
                    country = 'unknown'
                    
                    
                try:
                    qualify = report.find('primarysource')[1].text
                except:
                    qualify = '6'  # the qualify is unknown
                    
#                 qualify = report.find('primarysource')[1].text
                    
                if qualify not in {'1', '2', '3', '4', '5', '6','7'}:
                    qualify = '0'
                                      
                    
                try:
                    serious = report.find('serious').text
                except:
                    serious = '-1'
                
                try:
                    s_1 = report.find('seriousnessdeath').text
                except:
                    s_1 = '0'
                try:
                    s_2 = report.find('seriousnesslifethreatening').text
                except:
                    s_2 = '0'
                try:
                    s_3 = report.find('seriousnesshospitalization').text
                except:
                    s_3 = '0'
                try:
                    s_4 = report.find('seriousnessdisabling').text
                except:
                    s_4 = '0'
                try:
                    s_5 = report.find('seriousnesscongenitalanomali').text
                except:
                    s_5 = '0'
                try:
                    s_6 = report.find('seriousnessother').text
                except:
                    s_6 = '0'
                serious_subtype = [s_1, s_2, s_3, s_4, s_5, s_6]
            except:
                miss_admin +=1
                continue

            try:  # Optional information
                # receivedate: Date when the report was the FIRST received
                receivedateformat, receivedate = report.find('receivedateformat').text, report.find('receivedate').text
                receivedate = date_normalize(receivedateformat, receivedate)
            except:
                receivedate = '0'
            
            try:
                # receiptdate: Date of most RECENT report received
                receiptdateformat, receiptdate = report.find('receiptdateformat').text, report.find('receiptdate').text
                receiptdate = date_normalize(receiptdateformat, receiptdate)
            except:
                 receiptdate =  '0'

            for patient in report.findall('patient'):
                """Demographic Information"""                
                try:
                    age = patient.find('patientonsetage').text
                except:
                    age = -1 # unknown age
                try:
                    ageunit = patient.find('patientonsetageunit').text
                except:
                    ageunit = '801' 
                # normalize age
                try:
                    age = int(age)  
                    if age!= -1:
                        if ageunit == '800':  # Decade 
                            age = '-1'
                        elif ageunit == '801':  # Year
                            age = age
                        elif ageunit == '802':  # Month
                            age = int(age/12)
                        elif ageunit == '803':  # Week
                            age = int(age/52)
                        elif ageunit == '804':  # Day
                            age = int(age/365)
                        elif ageunit == '805':  # Hour
                            age = int(age/(24*365))
    #                     else:
    #                         age = '-1'  # unknown age
                except:
                    age = -1
                    
                      
                try:
                    gender = patient.find('patientsex').text
                except:
                    gender = '0'
                try:
                    weight = patient.find('patientweight').text
                except:
                    weight = '0'
                ## Nothing is mandatory
#                 if age == -1 and gender== '0':  # Mandatory: if age & gender both missing, ignore this report.
#                     miss_patient +=1
#                     continue

                reaction_list = []
                for side_ in patient.findall('reaction'):
                    try:  # outcome: 1-6, 6 levels in total
                        try: 
                            PT_code = side_[0].text
                        except:
                            PT_code = '0'
                        try:
                            outcome = side_[2].text
                        except:
                            outcome = '6'
                        try:
                            PT = side_[1].text
                        except:
                            PT = 'none'
                        reaction = [PT_code, PT, outcome]
                    except:
                        continue
                    reaction_list.append(reaction) 
                if reaction_list.__len__() == 0:  # Mandatory condition: at least has one reaction
                    miss_reaction += 1
                    continue

                drug_list = []
                for drug_ in patient.findall('drug'):
                    try:
                        try:
                            char =  drug_.find('drugcharacterization').text  # drugcharacterization: 1(suspect)/2(concomitant)/3(interacting)
                        except:
                            char = '0'
                        try:
                            product =  drug_.find('medicinalproduct').text  # drug brand
                        except:
                            product = 'none'
                        """Dosage are generally fixed according to the indication"""
                        try: 
                            dorse, unit=  drug_.find('drugstructuredosagenumb').text, drug_.find('drugstructuredosageunit').text
                            drugseparatedosagenumb, drugintervaldosageunitnumb, drugintervaldosagedefinition = \
                                drug_.find('drugseparatedosagenumb').text, drug_.find('drugintervaldosageunitnumb').text, \
                                drug_.find('drugintervaldosagedefinition').text
                            form = drug_.find('drugdosageform').text  # tablet or capsule or sth 
                        except:
                            dorse, unit, drugseparatedosagenumb,drugintervaldosageunitnumb, drugintervaldosagedefinition, form =\
                            '0', '0', '0','0','0', '0'
                        try:
                            route = drug_.find('drugadministrationroute').text
                            if route == '048':
                                route = '1'  # oral 
                            elif route == '061':
                                route = '2'  # Topical
                        except:
                            route = '0'  # no information of route
                        
                        try:
                            indication = drug_.find('drugindication').text  # indication (disease): super important
                        except:
                            indication = 'none'

                        try:
                            start_format, start_date = drug_.find('drugstartdateformat').text, drug_.find('drugstartdate').text
                            start_date = date_normalize(start_format, start_date)
                        except:
                            start_date = '0'
                        try:
                            end_format, end_date = drug_.find('drugenddateformat').text, drug_.find('drugenddate').text
                            end_date = date_normalize(end_format, end_date)
                        except:
                            try:
                                end_date = receiptdate
                            except:
                                end_date = '0'
                            
                        try:
                            action = drug_.find('actiondrug').text
                        except:
                            action = '5'
                        try:
                            additional = drug_.find('drugadditional').text
                        except:
                            additional = '3'
                        try:
                            readm = drug_.find('drugrecurreadministration').text
                        except:
                            readm = '3'
                        try:
                            substance = drug_.find('activesubstance')[0].text
                        except:
                            substance = 'none'
                    except:  # Mandatory condition: if none of the above information is provided, ignore this report
                        continue
                    drug = [char, product, dorse, unit, drugseparatedosagenumb, drugintervaldosageunitnumb,
                            drugintervaldosagedefinition, form, route, indication, start_date, end_date, action,
                            readm, additional, substance]
                    drug_list.append(drug)
                if drug_list.__len__() ==0:
                    miss_drug += 1
                    continue

                """for patient_ID"""
                dic[count] = [version, report_id, case_id, country, qualify, serious, 
                              s_1, s_2, s_3, s_4, s_5, s_6, 
                              receivedate, receiptdate,  
                              age, gender, weight, reaction_list, drug_list]
                count += 1

        pickle.dump(dic, open('../Data/parsed/'+ qtr_name+'.pk', 'wb'))
        
        n_reports.append(len(dic))
        print(qtr_name+' file saved. with', len(dic), 'reports')
        miss_count[qtr_name] = [nmb_reports, miss_admin, miss_patient, miss_reaction, miss_drug]

print ('All data saved')

I am parsing: 2013q1
find 1 files
['/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2013q1/xml/ADR13Q1.xml']
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2013q1/xml/ADR13Q1.xml


  0%|          | 0/223016 [00:00<?, ?it/s]

223017


100%|██████████| 223016/223016 [00:24<00:00, 9155.40it/s] 


2013q1 file saved. with 223016 reports
I am parsing: 2013q2
find 1 files
['/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2013q2/xml/ADR13Q2.xml']
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2013q2/xml/ADR13Q2.xml
171765


...

2020q1 file saved. with 460327 reports
I am parsing: 2020q2
find 3 files
['/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/1_ADR20Q2.xml', '/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/2_ADR20Q2.xml', '/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/3_ADR20Q2.xml']
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/1_ADR20Q2.xml
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/2_ADR20Q2.xml
finished merge /n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/2_ADR20Q2.xml
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/3_ADR20Q2.xml
finished merge /n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q2/XML/3_ADR20Q2.xml
429230


100%|██████████| 429227/429227 [00:43<00:00, 9831.74it/s] 


2020q2 file saved. with 429227 reports
I am parsing: 2020q3
find 3 files
['/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/1_ADR20Q3.xml', '/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/2_ADR20Q3.xml', '/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/3_ADR20Q3.xml']
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/1_ADR20Q3.xml
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/2_ADR20Q3.xml
finished merge /n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/2_ADR20Q3.xml
/n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/3_ADR20Q3.xml
finished merge /n/data1/hms/dbmi/zitnik/lab/datasets/2020-08-FAERS/data/2020q3/XML/3_ADR20Q3.xml
431670


100%|██████████| 431667/431667 [00:44<00:00, 9696.20it/s] 


2020q3 file saved. with 431667 reports
All data saved


## Sanity check

In [7]:
# check the number of reports in 2019 Q4
sep_2020 =pickle.load(open('../Data/parsed/2019q4.pk', 'rb'))
len(sep_2020)

419581

In [1]:
reports_pd = pd.DataFrame(sep_2020.values(), 
                          columns=['version','report_id','case_id','country','qualify','serious',
                                   's1','s2','s3','s4','s5','s6','receivedate','receiptdate',
                                   'age','gender','weight','SE','drugs'])

reports_pd.head(3)

,version,report_id,case_id,country,qualify,serious,s1,s2,s3,s4,s5,s6,receivedate,receiptdate,age,gender,weight,SE,drugs
0,2,10012352,US-PFIZER INC-2014070045,US,4,1,0,0,0,0,0,1,5186,7243,-1,0,0,"[[22.1, Type 2 diabetes mellitus, 6], [22.1, A...","[[1, LIPITOR, 80, 003, 1, 1, 804, FILM-COATED ..."
1,2,10015533,US-PFIZER INC-2014070019,US,4,1,0,0,0,0,0,1,5189,7243,56,0,0,"[[22.1, Type 2 diabetes mellitus, 6], [22.1, S...","[[1, LIPITOR, 0, 0, 0, 0, 0, 0, 0, CARDIOVASCU..."
2,2,10018621,US-PFIZER INC-2014070521,US,4,1,0,0,0,0,0,1,5190,7243,51,0,0,"[[22.1, Type 2 diabetes mellitus, 6]]","[[1, LIPITOR, 0, 0, 0, 0, 0, 0, 1, LOW DENSITY..."


In [5]:
for mm in ['1', '2', '3', '4', '5', '6','0', '7']:
    id_qua22w = [h ==mm for h in reports_pd.qualify ]
    reports_pd_sh = reports_pd[id_qua22w]  # professional: 1,2,3
    print('Focus on professional qualification, reports #',mm, reports_pd_sh.shape)

Focus on professional qualification, reports # 1 (93712, 19)
Focus on professional qualification, reports # 2 (31645, 19)
Focus on professional qualification, reports # 3 (97773, 19)
Focus on professional qualification, reports # 4 (4796, 19)
Focus on professional qualification, reports # 5 (170865, 19)
Focus on professional qualification, reports # 6 (20282, 19)
Focus on professional qualification, reports # 0 (508, 19)
Focus on professional qualification, reports # 7 (0, 19)


# Merge files 

In [12]:
ho_combos = {}

nmb = 0 
n_reports = []
for yr in range(2013, 2021):

    if yr == 2020:  
        qtr_list = [1, 2, 3]
    else:
        qtr_list=[1,2,3,4]
    for qtr in qtr_list:
        qtr_name = str(yr)+'q'+ str(qtr)
        file_path = '../Data/parsed/' +qtr_name +'.pk'
        print('I am loading {} from {}'.format(qtr_name, file_path))
        dic = pickle.load(open(file_path, 'rb'))
        n_reports.append(len(dic))
        print('loaded', len(dic))
        values = np.array(list(dic.values()))
        print(values.shape)   
        
        admin_demo = values[:, :17]
        # Find all reactions/side effects in this quarter
        se = values[:, 17] 
        drugs = values[:, 18]              
        for i in range(values.shape[0]): # dive into a single report
            new_se = []
            new_drug = [] # empty mapped se and drugs
            new_indication = [] 
            
            se_report = se[i] 
            drugs_report = drugs[i]
            for j in range(len(se_report)):  # dive into a single reaction
                se_reaction = se_report[j][1].lower() 
                if '\\' in se_reaction:
                    se_reaction = se_reaction.split('\\')[0] 
                if se_reaction in se_dic:
                    se_key = se_reaction
                elif ' ' in se_reaction:
                    se_key_sets = se_reaction.split(' ')
                    if se_key_sets[0] in se_dic:
                        se_key = se_key_sets[0]
                    elif se_key_sets[1] in se_dic:
                        se_key = se_key_sets[1]                      
                try: 
                    new_se.append(meddra_pd_all[se_reaction][0])   # Use MedDRA ID.
                except:  # if the key not in se_dic, continue
                    continue
                
            for k in range(len(drugs_report)):  # dive into a single drug
                drug = drugs_report[k][-1].lower()   # the drug substance
                indication = drugs_report[k][9].lower()  # the indication names
                
                
                # find the proper drug_key that exist in drug_dic
                if '\\' in drug:
                    drug = drug.split('\\')[0] 
                if drug in drug_dic:
                    drug_key = drug
                elif ' ' in drug:
                    key_sets = drug.split(' ')
                    if key_sets[0] in drug_dic:
                        drug_key = key_sets[0]

                    elif key_sets[1] in drug_dic:
                        drug_key = key_sets[1]               
                try:
#                     new_drug.append(drug_dic[drug_key][1])   # use code
                    new_drug.append(drug_dic[drug_key][0])     # use drugbank ID
                except:  # if the key not in drug_dic, continue
                    continue
                
                # find the proper drug_key that exist in drug_dic
                # Using meddra_pd_all
                if '\\' in indication:
                    indication = indication.split('\\')[0] 
                if indication in meddra_pd_all:
                    indication_key = indication
                elif ' ' in indication:
                    key_sets = indication.split(' ')
                    if key_sets[0] in meddra_pd_all:
                        indication_key = key_sets[0]

                    elif key_sets[1] in meddra_pd_all:
                        indication_key = key_sets[1]               
                try:
                    new_indication.append(meddra_pd_all[indication_key][0])     # use MedDRA ID
                except:  # if the key not in drug_dic, continue
                    continue

            # Feed into dictionary
            new_se.sort()
            new_drug.sort()  # keep the new_se and new_drug sorted, incase [1,2] and [2,1] appears as different drug list
            new_indication.sort()

            xx = list(admin_demo[i])
            xx.append(new_se)
            xx.append(new_drug)
            xx.append(new_indication)
            ho_combos[nmb] = xx   #[admin_demo[i], new_se, new_drug, new_indication]
            nmb += 1
            
print('#- all reports', sum(n_reports))
print('The No. of high order combos:',len(ho_combos))
pickle.dump(ho_combos, open('../Data/parsed/reports_v4.pk', 'wb'))
# Please revise 'Data/processed_2/v4/' to 'Data/parsed/'

print('ho_combos saved', ho_combos.get(0)) 

I am loading 2013q1 from ../Data/processed_2/v4/2013q1.pk
loaded 223016
(223016, 19)
I am loading 2013q2 from ../Data/processed_2/v4/2013q2.pk
loaded 171764
(171764, 19)
I am loading 2013q3 from ../Data/processed_2/v4/2013q3.pk
loaded 185569
(185569, 19)
I am loading 2013q4 from ../Data/processed_2/v4/2013q4.pk
loaded 232247
(232247, 19)
I am loading 2014q1 from ../Data/processed_2/v4/2014q1.pk
loaded 260057
(260057, 19)
I am loading 2014q2 from ../Data/processed_2/v4/2014q2.pk
loaded 223845
(223845, 19)
I am loading 2014q3 from ../Data/processed_2/v4/2014q3.pk
loaded 211308
(211308, 19)
I am loading 2014q4 from ../Data/processed_2/v4/2014q4.pk
loaded 207964
(207964, 19)
I am loading 2015q1 from ../Data/processed_2/v4/2015q1.pk
loaded 317071
(317071, 19)
I am loading 2015q2 from ../Data/processed_2/v4/2015q2.pk
loaded 289270
(289270, 19)
I am loading 2015q3 from ../Data/processed_2/v4/2015q3.pk
loaded 398860
(398860, 19)
I am loading 2015q4 from ../Data/processed_2/v4/2015q4.pk
loaded 

# Make DataFrame

## Remove duplicate case_id and report_id


In [3]:
reports_v4 = pickle.load(open('../Data/parsed/reports_v4.pk', 'rb'))
len(reports_v4) 

10443476

In [14]:
reports_pd = pd.DataFrame(reports_v4.values(), 
                          columns=['version','report_id','case_id','country','qualify','serious',
                                   's1','s2','s3','s4','s5','s6','receivedate','receiptdate',
                                   'age','gender','weight','SE','drugs','indications'])

reports_pd['lastingdays'] =  reports_pd['receiptdate'] -reports_pd['receivedate']

In [15]:
# check the #- of headache
ix = ['10019211' in j for j in reports_pd.SE]
print('#-of headache', sum(ix))
"""# of drugs and AEs"""
import itertools
n_se = len(set(list(itertools.chain(*reports_pd.SE))))
n_drug = len(set(list(itertools.chain(*reports_pd.drugs))))
print('#-SE: {}, #-drugs: {} in #-reports:{}'.format(n_se, n_drug, len(reports_pd)))

#-of headache 352703
#-SE: 19193, #-drugs: 3624 in #-reports:10443476


In [10]:
# this command is time-consumming. 
reports_pd['date'] = reports_pd.apply(lambda row: str(pd.Period('2000-01-01')+int(row['receivedate'])), axis = 1)
reports_pd.head()

,version,report_id,case_id,country,qualify,serious,s1,s2,s3,s4,...,receivedate,receiptdate,age,gender,weight,SE,drugs,indications,lastingdays,date
0,18,6161368,PHEH2006US13340,US,5,1,1,0,1,0,...,2498,4776,68,2,0,"[10001488, 10002034, 10002855, 10002948, 10003...",[],[],2278,2006-11-03
1,2,7820167,US-ROCHE-735882,US,4,1,0,0,1,0,...,4069,4764,39,2,68,"[10000081, 10003246, 10009900, 10012378, 10013...",[],[],695,2011-02-21
2,11,7683841,PHHY2009CA17701,CA,3,1,0,0,1,0,...,3982,4777,-1,2,0,"[10000081, 10000087, 10001949, 10003445, 10006...",[],[],795,2010-11-26
3,2,7684489,US-AMGEN-USASP2010000328,US,3,1,0,0,1,0,...,3985,4777,52,2,0,"[10021519, 10023424, 10025197, 10028694, 10038...",[],[],792,2010-11-29
4,2,7823627,PHHO2011US03018,US,3,1,0,0,0,0,...,4071,4783,57,2,56,[10064658],[],[],712,2011-02-23


In [11]:
print('#-of all reports',reports_pd.shape)
reports_pd = reports_pd.drop_duplicates(['report_id', 'case_id','receivedate'], keep="last")
print('After remove duplicate reports',reports_pd.shape)

# reports_pd = reports_pd.drop_duplicates(['case_id'], keep="last")
# print('After remove duplicate case_id',reports_pd.shape)

#-of all reports (10443476, 22)
After remove duplicate reports (9325731, 22)


In [12]:
# check the #- of headache
ix = ['10019211' in j for j in reports_pd.SE]
print('#-of headache', sum(ix))

#-of headache 300301


In [13]:
pickle.dump(reports_pd, open('../Data/parsed/reports_v4_pd_new.pk', 'wb'))
print('reports_v4_pd_new saved', reports_pd.shape) 

reports_v4_pd_new saved (9325731, 22)


##  Add receiptdate

In [9]:
new_pd = pickle.load(open('../Data/parsed/reports_v4_pd_new.pk', 'rb'))
print('reports_v4_pd_new saved', new_pd.shape) 

reports_v4_pd_new saved (9325731, 22)


In [10]:
new_pd['receipt_date'] = new_pd.apply(lambda row: str(pd.Period('2000-01-01')+int(row['receiptdate'])), axis = 1)
new_pd.head()

,version,report_id,case_id,country,qualify,serious,s1,s2,s3,s4,...,receiptdate,age,gender,weight,SE,drugs,indications,lastingdays,date,receipt_date
0,18,6161368,PHEH2006US13340,US,5,1,1,0,1,0,...,4776,68,2,0,"[10001488, 10002034, 10002855, 10002948, 10003...",[],[],2278,2006-11-03,2013-01-28
1,2,7820167,US-ROCHE-735882,US,4,1,0,0,1,0,...,4764,39,2,68,"[10000081, 10003246, 10009900, 10012378, 10013...",[],[],695,2011-02-21,2013-01-16
2,11,7683841,PHHY2009CA17701,CA,3,1,0,0,1,0,...,4777,-1,2,0,"[10000081, 10000087, 10001949, 10003445, 10006...",[],[],795,2010-11-26,2013-01-29
3,2,7684489,US-AMGEN-USASP2010000328,US,3,1,0,0,1,0,...,4777,52,2,0,"[10021519, 10023424, 10025197, 10028694, 10038...",[],[],792,2010-11-29,2013-01-29
4,2,7823627,PHHO2011US03018,US,3,1,0,0,0,0,...,4783,57,2,56,[10064658],[],[],712,2011-02-23,2013-02-04


In [11]:
pickle.dump(new_pd, open('../Data/curated/patient_safety.pk', 'wb'))

print('patient_safety saved', new_pd.shape) 

patient_safety saved (9325731, 23)
